<a href="https://colab.research.google.com/github/sydstewart/Orders/blob/main/Impsim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [40]:
"""Companion code to https://realpython.com/simulation-with-simpy/
'Simulating Real-World Processes With SimPy'
Python version: 3.7.3
SimPy version: 3.0.11
"""
!pip install simpy
import simpy
import random
import statistics
import numpy as np

wait_times = []
kitwork_times = []

class Implement(object):
    def __init__(self, env, num_kitters):
        self.env = env
        self.kitter = simpy.Resource(env, num_kitters)
        # self.server = simpy.Resource(env, num_servers)
        # self.usher = simpy.Resource(env, num_ushers)

    def kitwork(self, project):
        yield self.env.timeout(random.randint(1, 6))

    def kitwait(self, project):
        yield self.env.timeout(random.randint(20, 80))
    
    # def reqwork(self, project):
    #     yield self.env.timeout(random.randint(1, 6))

    # def reqwait(self, project):
    #     yield self.env.timeout(random.randint(20, 80))

    # def check_ticket(self, project):
    #     yield self.env.timeout(3 / 60)

    # def sell_food(self, project):
    #     yield self.env.timeout(random.randint(1, 5))


def go_to_implement(env, project, implementation):
    # Moviegoer arrives at the implementation
    arrival_time = env.now

    with implementation.kitter.request() as request:
        yield request
        yield env.process(implementation.kitwork(project))
    kitwork_times.append(env.now - arrival_time)

    with implementation.kitter.request() as request:
        yield request
        yield env.process(implementation.kitwait(project))

    # with implementation.kitter.request() as request:
    #     yield request
    #     yield env.process(implementation.reqwork(project))

    # with implementation.kitter.request() as request:
    #     yield request
    #     yield env.process(implementation.reqwait(project))

    # with implementation.usher.request() as request:
    #     yield request
    #     yield env.process(implementation.check_ticket(project))

    # if random.choice([True, False]):
    #     with implementation.server.request() as request:
    #         yield request
    #         yield env.process(implementation.sell_food(project))

    # Moviegoer heads into the implementation
    wait_times.append(env.now - arrival_time)


def run_implementation(env, num_kitters):
    implementation = Implement( env, num_kitters)

    for project in range(3):
        env.process(go_to_implement(env, project, implementation))

    while True:
        yield env.timeout(0.20)  # Wait a bit before generating a new project

        project += 1
        env.process(go_to_implement(env, project, implementation))


def get_average_wait_time(wait_times):
    average_hours = statistics.mean(wait_times)
    average_days = average_hours/8
    # Pretty print the results
    # minutes, frac_minutes = divmod(average_wait, 1)
    # seconds = frac_minutes * 60
    return average_days #round(minutes), round(seconds)


# def get_user_input():
#     num_kitters = input("Input # of kitters working: ")
#     num_servers = input("Input # of servers working: ")
#     num_ushers = input("Input # of ushers working: ")
#     params = [num_kitters, num_servers, num_ushers]
#     if all(str(i).isdigit() for i in params):  # Check input is valid
#         params = [int(x) for x in params]
#     else:
#         print(
#             "Could not parse input. Simulation will use default values:",
#             "\n1 kitter, 1 server, 1 usher.",
#         )
#         params = [1, 1, 1]
#     return params


def main():
    # Setup
    random.seed(142)
    num_kitters = 1 #, num_servers, num_ushers = get_user_input()
    # num_reqs = 1
    # Run the simulation
    env = simpy.Environment()
    env.process(run_implementation(env, num_kitters))
    env.run(until=9000)

    # View the results
    average_days = get_average_wait_time(wait_times)
    print(
        "Running simulation...",
        f"\nThe average wait time is {average_days} days",
    )
    print("==================wait_times =============================")
    print(wait_times)
    print("==================kitwork_times =============================")
    print(kitwork_times)
    counts, bins = np.histogram(wait_times)
    print(bins)
    print(counts)
if __name__ == "__main__":
    main()

Running simulation... 
The average wait time is 528.6625 days
==================wait_times =============================
[137, 311, 420, 606.8, 737.6, 821.4, 932.2, 1056.0, 1146.8, 1254.6, 1351.4, 1397.2, 1453.0, 1583.8, 1660.6, 1784.4, 1891.2, 1989.0, 2135.8, 2271.6, 2311.4, 2401.2, 2494.0, 2589.8, 2756.6, 2884.4, 3022.2, 3806.0, 3886.8, 3998.6, 4141.4, 4190.2, 4305.0, 4372.8, 4535.6, 4581.4, 4691.2, 4824.0, 4928.8, 5070.6, 5187.4, 5307.2, 5366.0, 5487.8, 5658.6, 5760.4, 5906.2, 6013.0, 6068.8, 6131.6, 6293.4, 6363.2, 6483.0, 6537.8, 6614.6, 6739.4, 6825.2, 6908.0, 8269.8, 8329.6, 8462.4, 8592.2, 8663.0, 8726.8, 8783.6, 8921.4]
==================kitwork_times =============================
[5, 11, 17, 22.8, 26.6, 28.4, 32.2, 35.0, 37.8, 40.6, 43.4, 44.2, 45.0, 47.8, 49.6, 54.4, 59.2, 61.0, 65.8, 70.6, 71.4, 75.2, 77.0, 77.8, 83.6, 86.4, 91.2, 133.0, 133.8, 137.6, 143.4, 144.2, 147.0, 148.8, 154.6, 155.4, 159.2, 163.0, 164.79999999999998, 170.6, 173.4, 177.2, 178.0, 180.8, 186.6, 189.4,